In [1]:
import numpy as np
import pandas as pd
from scipy import stats
from scipy.stats import norm
from tqdm import tqdm 

In [87]:
df = pd.read_csv("features.csv")
df 

,id,eye,stimuli,color,glaucoma,mean_window_50_0,std_window_50_0,median_window_50_0,max_window_50_0,min_window_50_0,...,std_window_50_54,median_window_50_54,max_window_50_54,min_window_50_54,range_window_50_54,IQR_window_50_54,Q1_window_50_54,Q3_window_50_54,kurtosis_window_50_54,skewness_window_50_54
0,C001,left,ramp,white,no,32.233702,21.220448,29.165282,60.572493,0.000000,...,17.153265,55.027706,68.376038,0.000000,68.376038,1.782344,53.771554,55.553898,1.251011,-1.684972
1,C001,left,ramp,blue,no,54.801834,7.457179,56.462238,57.416459,13.402000,...,6.083763,54.958326,56.582669,23.579367,33.003302,1.043700,54.714285,55.757985,19.538668,-4.602760
2,C001,left,ramp,green,no,55.186039,5.784269,55.880359,68.521255,20.023360,...,5.171902,56.066412,61.083622,26.585171,34.498452,0.714797,55.668923,56.383720,18.591960,-4.266838
3,C001,left,flash,white,no,50.753054,15.557146,56.509845,80.000000,11.077743,...,6.081288,56.265699,59.186466,13.580236,45.606230,0.775210,55.944968,56.720178,41.859989,-6.504073
4,C001,left,flash,blue,no,47.545593,15.580917,55.304546,80.000000,10.388611,...,15.036102,55.169589,80.000000,12.908037,67.091963,2.188269,54.205255,56.393523,1.130352,-1.465107
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
651,S020,right,flash,red,yes,48.162405,10.844362,51.866347,62.190289,11.731321,...,2.518823,50.626490,53.421669,40.842110,12.579559,1.261606,50.382777,51.644383,7.504337,-2.672959
652,S020,right,ramp,red,yes,51.051313,2.988132,51.567226,55.032780,37.716240,...,2.742470,49.961193,51.834238,37.091038,14.743200,2.602026,48.123849,50.725875,7.535957,-2.569689
653,S020,right,flash,white,yes,51.057405,7.632710,52.392627,53.833977,0.000000,...,16.909001,51.779498,58.379410,0.000000,58.379410,11.478749,41.723887,53.202636,1.894031,-1.819491
654,S020,right,ramp,green,yes,26.944068,23.689560,36.986803,54.678856,0.000000,...,0.646717,49.818981,50.948158,48.174379,2.773779,1.128346,49.218899,50.347245,-0.775603,-0.194488


In [88]:
df.dtypes

id                        object
eye                       object
stimuli                   object
color                     object
glaucoma                  object
                          ...   
IQR_window_50_54         float64
Q1_window_50_54          float64
Q3_window_50_54          float64
kurtosis_window_50_54    float64
skewness_window_50_54    float64
Length: 610, dtype: object

In [199]:
print(df.columns.tolist())

['id', 'eye', 'stimuli', 'color', 'glaucoma', 'mean_window_50_0', 'std_window_50_0', 'median_window_50_0', 'max_window_50_0', 'min_window_50_0', 'range_window_50_0', 'IQR_window_50_0', 'Q1_window_50_0', 'Q3_window_50_0', 'kurtosis_window_50_0', 'skewness_window_50_0', 'mean_window_50_1', 'std_window_50_1', 'median_window_50_1', 'max_window_50_1', 'min_window_50_1', 'range_window_50_1', 'IQR_window_50_1', 'Q1_window_50_1', 'Q3_window_50_1', 'kurtosis_window_50_1', 'skewness_window_50_1', 'mean_window_50_2', 'std_window_50_2', 'median_window_50_2', 'max_window_50_2', 'min_window_50_2', 'range_window_50_2', 'IQR_window_50_2', 'Q1_window_50_2', 'Q3_window_50_2', 'kurtosis_window_50_2', 'skewness_window_50_2', 'mean_window_50_3', 'std_window_50_3', 'median_window_50_3', 'max_window_50_3', 'min_window_50_3', 'range_window_50_3', 'IQR_window_50_3', 'Q1_window_50_3', 'Q3_window_50_3', 'kurtosis_window_50_3', 'skewness_window_50_3', 'mean_window_50_4', 'std_window_50_4', 'median_window_50_4', '

In [200]:
# Lista de columnas adicionales que quieres conservar
additional_columns = ['id', 'eye', 'stimuli', 'color', 'glaucoma']

# Seleccionar las columnas que van desde 50_0 hasta 50_14
pattern_columns = [col for col in df.columns if '50_' in col and int(col.split('_')[-1]) <= 14]

# Combinar las columnas seleccionadas con las columnas adicionales
selected_columns = additional_columns + pattern_columns

# Crear un nuevo dataframe con las columnas seleccionadas
df_selected = df[selected_columns]

# Mostrar las primeras filas del dataframe seleccionado
df_selected.head()

,id,eye,stimuli,color,glaucoma,mean_window_50_0,std_window_50_0,median_window_50_0,max_window_50_0,min_window_50_0,...,Q3_window_50_13,mean_window_50_14,std_window_50_14,median_window_50_14,max_window_50_14,min_window_50_14,range_window_50_14,IQR_window_50_14,Q1_window_50_14,Q3_window_50_14
0,C001,left,ramp,white,0,32.233702,21.220448,29.165282,60.572493,0.000000,...,31.493153,32.542614,1.027304,32.249439,34.517437,31.136570,3.380867,1.514904,31.697360,33.212264
1,C001,left,ramp,blue,0,54.801834,7.457179,56.462238,57.416459,13.402000,...,25.610007,24.180124,3.738143,25.026102,25.862329,0.000000,25.862329,0.644458,24.658390,25.302848
2,C001,left,ramp,green,0,55.186039,5.784269,55.880359,68.521255,20.023360,...,34.850234,34.544660,0.699974,34.629436,36.062810,33.197536,2.865274,1.018261,34.093526,35.111786
3,C001,left,flash,white,0,50.753054,15.557146,56.509845,80.000000,11.077743,...,55.207252,53.163536,7.964327,55.013068,69.071053,14.750408,54.320645,0.643071,54.614230,55.257301
4,C001,left,flash,blue,0,47.545593,15.580917,55.304546,80.000000,10.388611,...,51.471113,43.186415,14.899464,51.411627,58.071033,10.783411,47.287623,11.282030,41.198513,52.480543


In [163]:
total_nulls = df.isnull().sum().sum()
print(f"Total de valores nulos en el dataset: {total_nulls}")

Total de valores nulos en el dataset: 16


In [164]:
# Verificar si el DataFrame tiene valores nulos
def check_nulls(dataset):
    nulls = dataset.isnull().sum()
    print("Valores nulos por columna:")
    print(nulls[nulls > 0])  # Mostrar solo las columnas con valores nulos
    return nulls[nulls > 0]

nulls = check_nulls(df)

Valores nulos por columna:
kurtosis_window_50_12    1
skewness_window_50_12    1
kurtosis_window_50_13    1
skewness_window_50_13    1
kurtosis_window_50_14    1
skewness_window_50_14    1
kurtosis_window_50_35    1
skewness_window_50_35    1
kurtosis_window_50_41    1
skewness_window_50_41    1
kurtosis_window_50_47    1
skewness_window_50_47    1
kurtosis_window_50_53    1
skewness_window_50_53    1
kurtosis_window_50_54    1
skewness_window_50_54    1
dtype: int64


In [165]:
# Eliminar columnas que contienen cualquier valor NaN
df = df.dropna(axis=1)
nulls = check_nulls(df)

Valores nulos por columna:
Series([], dtype: int64)


In [201]:
# Eliminar columnas que contienen cualquier valor NaN
df_selected = df_selected.dropna(axis=1)
nulls = check_nulls(df_selected)

Valores nulos por columna:
Series([], dtype: int64)


In [166]:
pd.set_option('future.no_silent_downcasting', True)
df = df.replace({'yes': 1, 'no': 0})

# Convertir la columna 'glaucoma' a tipo numérico
df['glaucoma'] = df['glaucoma'].astype(int)

# Verificar el tipo de dato de la columna
print(df.dtypes['glaucoma'])

int32


In [202]:
pd.set_option('future.no_silent_downcasting', True)
df_selected = df_selected.replace({'yes': 1, 'no': 0})

# Convertir la columna 'glaucoma' a tipo numérico
df_selected['glaucoma'] = df_selected['glaucoma'].astype(int)

# Verificar el tipo de dato de la columna
print(df_selected.dtypes['glaucoma'])

int32


In [167]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Supongamos que tienes un dataset cargado
# dataset = pd.read_csv('path_to_your_dataset.csv')

# Primero, obtener los ids únicos
unique_ids = df['id'].unique()

# Realizar el train_test_split a nivel de id
train_ids, test_ids = train_test_split(unique_ids, test_size=0.2, random_state=42)

# Filtrar el df original usando los ids seleccionados
train_set = df[df['id'].isin(train_ids)]
test_set = df[df['id'].isin(test_ids)]

# Verifica los tamaños de los conjuntos
print(f"Tamaño del conjunto de entrenamiento: {len(train_set)}")
print(f"Tamaño del conjunto de prueba: {len(test_set)}")

Tamaño del conjunto de entrenamiento: 512
Tamaño del conjunto de prueba: 144


In [203]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Supongamos que tienes un dataset cargado
# dataset = pd.read_csv('path_to_your_dataset.csv')

# Primero, obtener los ids únicos
unique_ids = df_selected['id'].unique()

# Realizar el train_test_split a nivel de id
train_ids, test_ids = train_test_split(unique_ids, test_size=0.2, random_state=42)

# Filtrar el df_selected original usando los ids seleccionados
train_set = df_selected[df_selected['id'].isin(train_ids)]
test_set = df_selected[df_selected['id'].isin(test_ids)]

# Verifica los tamaños de los conjuntos
print(f"Tamaño del conjunto de entrenamiento: {len(train_set)}")
print(f"Tamaño del conjunto de prueba: {len(test_set)}")

Tamaño del conjunto de entrenamiento: 512
Tamaño del conjunto de prueba: 144


In [204]:
y_train = train_set.loc[:, 'glaucoma']
y_test = test_set.loc[:, 'glaucoma']
labels = df.loc[:, 'glaucoma']

In [205]:
print(y_train)
print(y_test)

0      0
1      0
2      0
3      0
4      0
      ..
619    1
620    1
621    1
622    1
623    1
Name: glaucoma, Length: 512, dtype: int32
64     0
65     0
66     0
67     0
68     0
      ..
651    1
652    1
653    1
654    1
655    1
Name: glaucoma, Length: 144, dtype: int32


In [170]:
# distribución original
labels.value_counts() / labels.count() * 100

glaucoma
0    51.219512
1    48.780488
Name: count, dtype: float64

In [171]:
# conjunto de entrenamiento
y_train.value_counts() / y_train.count() * 100

glaucoma
0    53.125
1    46.875
Name: count, dtype: float64

In [172]:
# conjunto de pruebas
y_test.value_counts() / y_test.count() * 100

glaucoma
1    55.555556
0    44.444444
Name: count, dtype: float64

In [173]:
df 

,id,eye,stimuli,color,glaucoma,mean_window_50_0,std_window_50_0,median_window_50_0,max_window_50_0,min_window_50_0,...,Q3_window_50_53,mean_window_50_54,std_window_50_54,median_window_50_54,max_window_50_54,min_window_50_54,range_window_50_54,IQR_window_50_54,Q1_window_50_54,Q3_window_50_54
0,C001,left,ramp,white,0,32.233702,21.220448,29.165282,60.572493,0.000000,...,55.863291,47.329627,17.153265,55.027706,68.376038,0.000000,68.376038,1.782344,53.771554,55.553898
1,C001,left,ramp,blue,0,54.801834,7.457179,56.462238,57.416459,13.402000,...,54.990369,54.026772,6.083763,54.958326,56.582669,23.579367,33.003302,1.043700,54.714285,55.757985
2,C001,left,ramp,green,0,55.186039,5.784269,55.880359,68.521255,20.023360,...,55.954966,54.948466,5.171902,56.066412,61.083622,26.585171,34.498452,0.714797,55.668923,56.383720
3,C001,left,flash,white,0,50.753054,15.557146,56.509845,80.000000,11.077743,...,57.017467,55.428993,6.081288,56.265699,59.186466,13.580236,45.606230,0.775210,55.944968,56.720178
4,C001,left,flash,blue,0,47.545593,15.580917,55.304546,80.000000,10.388611,...,56.715728,49.484305,15.036102,55.169589,80.000000,12.908037,67.091963,2.188269,54.205255,56.393523
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
651,S020,right,flash,red,1,48.162405,10.844362,51.866347,62.190289,11.731321,...,51.018237,50.494241,2.518823,50.626490,53.421669,40.842110,12.579559,1.261606,50.382777,51.644383
652,S020,right,ramp,red,1,51.051313,2.988132,51.567226,55.032780,37.716240,...,50.496840,49.084546,2.742470,49.961193,51.834238,37.091038,14.743200,2.602026,48.123849,50.725875
653,S020,right,flash,white,1,51.057405,7.632710,52.392627,53.833977,0.000000,...,52.670460,43.835050,16.909001,51.779498,58.379410,0.000000,58.379410,11.478749,41.723887,53.202636
654,S020,right,ramp,green,1,26.944068,23.689560,36.986803,54.678856,0.000000,...,50.267288,49.787670,0.646717,49.818981,50.948158,48.174379,2.773779,1.128346,49.218899,50.347245


In [174]:
df.dtypes

id                     object
eye                    object
stimuli                object
color                  object
glaucoma                int32
                       ...   
min_window_50_54      float64
range_window_50_54    float64
IQR_window_50_54      float64
Q1_window_50_54       float64
Q3_window_50_54       float64
Length: 594, dtype: object

In [206]:
# Hacer copias de train_set y test_set sin las columnas 'id' y 'glaucoma'
X_train = train_set.drop(columns=['id', 'glaucoma']).copy()
X_test = test_set.drop(columns=['id', 'glaucoma']).copy()

# Verifica los resultados
X_train.head()
X_test.head()

,eye,stimuli,color,mean_window_50_0,std_window_50_0,median_window_50_0,max_window_50_0,min_window_50_0,range_window_50_0,IQR_window_50_0,...,Q3_window_50_13,mean_window_50_14,std_window_50_14,median_window_50_14,max_window_50_14,min_window_50_14,range_window_50_14,IQR_window_50_14,Q1_window_50_14,Q3_window_50_14
64,left,ramp,blue,41.157644,5.676886,40.499168,80.000000,35.206100,44.793900,0.269042,...,43.432658,35.711826,16.653479,31.660171,80.000000,0.000000,80.000000,14.640245,26.488603,41.128848
65,left,ramp,white,38.272979,5.446160,37.868977,75.489609,31.918873,43.570736,0.385221,...,33.908146,33.216046,0.396214,33.187731,34.369762,32.545191,1.824572,0.488986,32.900018,33.389004
66,left,flash,white,37.639543,6.354851,36.783945,80.000000,24.832289,55.167711,0.995728,...,38.300826,39.198785,6.268935,38.285043,76.780052,34.765339,42.014713,0.589893,37.863660,38.453553
67,left,flash,blue,38.108122,2.607544,38.602344,45.919888,26.073679,19.846209,0.331165,...,38.112687,37.970424,1.109968,38.064829,42.873744,33.600101,9.273642,0.460184,37.835157,38.295341
68,left,ramp,green,38.101671,3.620865,38.750172,40.275343,13.650887,26.624457,0.497655,...,36.331841,36.354340,0.341005,36.260054,37.011776,35.856695,1.155081,0.574947,36.079443,36.654389


In [207]:
numeric = X_train.select_dtypes(["float64", "int32"]).columns
categorical = X_train.select_dtypes(["object"]).columns

In [177]:
print(categorical)

Index(['eye', 'stimuli', 'color'], dtype='object')


In [208]:
print(numeric)

Index(['mean_window_50_0', 'std_window_50_0', 'median_window_50_0',
       'max_window_50_0', 'min_window_50_0', 'range_window_50_0',
       'IQR_window_50_0', 'Q1_window_50_0', 'Q3_window_50_0',
       'kurtosis_window_50_0',
       ...
       'Q3_window_50_13', 'mean_window_50_14', 'std_window_50_14',
       'median_window_50_14', 'max_window_50_14', 'min_window_50_14',
       'range_window_50_14', 'IQR_window_50_14', 'Q1_window_50_14',
       'Q3_window_50_14'],
      dtype='object', length=159)


In [209]:
import pandas as pd
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Definir las columnas numéricas y categóricas
numeric = X_train.select_dtypes(["float64", "int32"]).columns
categorical = X_train.select_dtypes(["object"]).columns

# Pipelines para columnas numéricas y categóricas
numeric_transformations = Pipeline([
    ('scaler', MinMaxScaler())  # Escalado MinMax para columnas numéricas
])

categoric_transformations = Pipeline([
    ('category_one_hot', OneHotEncoder(sparse_output=False, handle_unknown="ignore"))  # Codificación OneHot para categóricas
])

# Preprocessor: combinación de los pipelines numéricos y categóricos
preprocessor = ColumnTransformer(transformers=[
    ('numerical', numeric_transformations, numeric),
    ('categorical', categoric_transformations, categorical)
])

In [210]:
# Aplicar el preprocessor (ajustar y transformar el conjunto de entrenamiento)
transformed_data = preprocessor.fit_transform(X_train)

# Obtener las categorías nuevas creadas por el OneHotEncoder
onehot_columns = preprocessor.named_transformers_['categorical'].named_steps['category_one_hot'].get_feature_names_out(categorical)

# Crear las columnas finales: numéricas + categóricas codificadas
final_columns = list(numeric) + list(onehot_columns)

# Convertir los datos transformados en un DataFrame para visualizar
transformed_df = pd.DataFrame(transformed_data, columns=final_columns)

# Ver las primeras filas del DataFrame transformado
transformed_df.head()

,mean_window_50_0,std_window_50_0,median_window_50_0,max_window_50_0,min_window_50_0,range_window_50_0,IQR_window_50_0,Q1_window_50_0,Q3_window_50_0,kurtosis_window_50_0,...,Q1_window_50_14,Q3_window_50_14,eye_left,eye_right,stimuli_flash,stimuli_ramp,color_blue,color_green,color_red,color_white
0,0.504571,0.897257,0.446006,0.757156,0.000000,0.759395,0.850318,0.224362,0.878695,0.010885,...,0.533639,0.551920,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,0.857842,0.315309,0.863441,0.717706,0.211410,0.551807,0.010737,0.866187,0.857848,0.517426,...,0.415135,0.420482,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
2,0.863857,0.244574,0.854542,0.856516,0.315859,0.608016,0.027741,0.853721,0.858794,0.590714,...,0.573979,0.583487,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
3,0.794465,0.657797,0.864169,1.000000,0.174746,0.864075,0.022895,0.864677,0.865785,0.082044,...,0.919454,0.918264,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,0.744257,0.658802,0.845737,1.000000,0.163875,0.872715,0.333374,0.618381,0.864718,0.052179,...,0.693595,0.872120,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0


In [211]:
X_train.head()

,eye,stimuli,color,mean_window_50_0,std_window_50_0,median_window_50_0,max_window_50_0,min_window_50_0,range_window_50_0,IQR_window_50_0,...,Q3_window_50_13,mean_window_50_14,std_window_50_14,median_window_50_14,max_window_50_14,min_window_50_14,range_window_50_14,IQR_window_50_14,Q1_window_50_14,Q3_window_50_14
0,left,ramp,white,32.233702,21.220448,29.165282,60.572493,0.000000,60.572493,43.426828,...,31.493153,32.542614,1.027304,32.249439,34.517437,31.136570,3.380867,1.514904,31.697360,33.212264
1,left,ramp,blue,54.801834,7.457179,56.462238,57.416459,13.402000,44.014458,0.548367,...,25.610007,24.180124,3.738143,25.026102,25.862329,0.000000,25.862329,0.644458,24.658390,25.302848
2,left,ramp,green,55.186039,5.784269,55.880359,68.521255,20.023360,48.497896,1.416761,...,34.850234,34.544660,0.699974,34.629436,36.062810,33.197536,2.865274,1.018261,34.093526,35.111786
3,left,flash,white,50.753054,15.557146,56.509845,80.000000,11.077743,68.922257,1.169259,...,55.207252,53.163536,7.964327,55.013068,69.071053,14.750408,54.320645,0.643071,54.614230,55.257301
4,left,flash,blue,47.545593,15.580917,55.304546,80.000000,10.388611,69.611389,17.025806,...,51.471113,43.186415,14.899464,51.411627,58.071033,10.783411,47.287623,11.282030,41.198513,52.480543


In [212]:
total_nulls = X_train.isnull().sum().sum()
print(f"Total de valores nulos en el dataset: {total_nulls}")

Total de valores nulos en el dataset: 0


## Baseline

In [107]:
from sklearn.model_selection import cross_val_score
from sklearn.dummy import DummyClassifier  # Ejemplo con un modelo Dummy (puedes reemplazar con otros modelos)
from sklearn.model_selection import cross_validate

# Definir el modelo Dummy para probar el ciclo (puedes cambiar por otro clasificador)
dummy_model = DummyClassifier(strategy="most_frequent")

# Crear el pipeline con el preprocesador y el modelo
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', dummy_model)  # Reemplaza esto con cualquier modelo que desees usar
])

# Definir las métricas a calcular
scoring = ['f1_macro', 'accuracy', 'roc_auc']

# Validación cruzada con múltiples métricas
cv_results = cross_validate(pipeline, X_train, y_train, cv=5, scoring=scoring, return_train_score=True)

# Mostrar resultados
print("F1-Score por fold:", cv_results['test_f1_macro'])
print("ROC-AUC por fold:", cv_results['test_roc_auc'])
print("Exactitud por fold:", cv_results['test_accuracy'])
print("Promedio F1-Score:", cv_results['test_f1_macro'].mean())

F1-Score por fold: [0.34810127 0.34810127 0.34615385 0.34615385 0.34615385]
ROC-AUC por fold: [0.5 0.5 0.5 0.5 0.5]
Exactitud por fold: [0.53398058 0.53398058 0.52941176 0.52941176 0.52941176]
Promedio F1-Score: 0.3469328140214216


In [183]:
from sklearn.metrics import f1_score, precision_recall_curve, average_precision_score, roc_auc_score
from sklearn.metrics import classification_report, confusion_matrix
import time
from sklearn.metrics import auc

def train_and_evaluate(pipe, X_train, y_train, X_test, y_test, print_=True):
    start_time = time.time()  # Start the timer

    pipe.fit(X_train, y_train)  # Entrenamiento

    end_time = time.time()  # Stop the timer
    training_time = end_time - start_time  # Calculate the elapsed time

    # Predicciones
    y_pred = pipe.predict(X_test)
    y_prob = pipe.predict_proba(X_test)[:, 1]  # Probabilidades para la clase positiva

    # Cálculo del AUC-PR usando probabilidades
    precision, recall, thresholds = precision_recall_curve(y_test, y_prob)
    auc_pr = auc(recall, precision)
    
    # Cálculo del AUC-ROC (si es relevante)
    auc_roc = roc_auc_score(y_test, y_prob)

    if print_:
        print("Matriz de confusión: \n")
        print(confusion_matrix(y_test, y_pred, labels=pipe.classes_))
        print("\nReporte de Clasificación: \n")
        print(classification_report(y_test, y_pred, target_names=[str(cls) for cls in pipe.classes_]))
        print(f"Tiempo de entrenamiento: {training_time:.2f} segundos")

In [213]:
from sklearn.dummy import DummyClassifier

dummy_pipe = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        ("Dummy", DummyClassifier(strategy="stratified")),
    ]
)

train_and_evaluate(dummy_pipe, X_train, y_train, X_test, y_test, print_=True)

Matriz de confusión: 

[[36 28]
 [51 29]]

Reporte de Clasificación: 

              precision    recall  f1-score   support

           0       0.41      0.56      0.48        64
           1       0.51      0.36      0.42        80

    accuracy                           0.45       144
   macro avg       0.46      0.46      0.45       144
weighted avg       0.47      0.45      0.45       144

Tiempo de entrenamiento: 0.01 segundos


## Modelos

### XGboost

In [214]:
from xgboost import XGBClassifier

# Definir el modelo XGBoost
xgb_model = XGBClassifier(n_estimators=200, learning_rate=0.1, max_depth=3, eval_metric='logloss')

# Definir el pipeline con el preprocesador y el modelo XGBoost
pipe = Pipeline([
    ('preprocessor', preprocessor),  # Preprocesador que creaste anteriormente
    ('classifier', xgb_model)        # Modelo de clasificación XGBoost
])

train_and_evaluate(pipe, X_train, y_train, X_test, y_test, print_=True)

Matriz de confusión: 

[[17 47]
 [43 37]]

Reporte de Clasificación: 

              precision    recall  f1-score   support

           0       0.28      0.27      0.27        64
           1       0.44      0.46      0.45        80

    accuracy                           0.38       144
   macro avg       0.36      0.36      0.36       144
weighted avg       0.37      0.38      0.37       144

Tiempo de entrenamiento: 0.56 segundos


In [217]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier

# Definir el modelo XGBoost
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')

# Definir el pipeline con el preprocesador y el modelo XGBoost
pipe = Pipeline([
    ('preprocessor', preprocessor),  # Preprocesador que creaste anteriormente
    ('classifier', xgb_model)        # Modelo de clasificación XGBoost
])

# Definir el espacio de búsqueda para los hiperparámetros
param_grid = {
    'classifier__n_estimators': [100, 200, 300],  # Número de árboles
    'classifier__learning_rate': [0.001, 0.01, 0.1],  # Tasa de aprendizaje
    'classifier__max_depth': [3, 5, 10, 25, 50, None],  # Profundidad máxima del árbol
    #'classifier__subsample': [0.8, 1.0],  # Subconjunto de muestras
    #'classifier__colsample_bytree': [0.8, 1.0],  # Subconjunto de características
}

# Crear el objeto GridSearchCV
grid_search = GridSearchCV(pipe, param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=2)

# Entrenar el GridSearchCV para encontrar los mejores hiperparámetros
grid_search.fit(X_train, y_train)

# Mostrar los mejores hiperparámetros encontrados
print("Mejores hiperparámetros encontrados:")
print(grid_search.best_params_)

# Evaluar el modelo con los mejores hiperparámetros en el conjunto de prueba
train_and_evaluate(grid_search.best_estimator_, X_train, y_train, X_test, y_test, print_=True)


Fitting 5 folds for each of 54 candidates, totalling 270 fits


c:\Users\Manel\miniconda3\envs\prepro\Lib\site-packages\xgboost\core.py:158: UserWarning:

[13:03:33] WARNING: D:\bld\xgboost-split_1724807710452\work\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.




Mejores hiperparámetros encontrados:
{'classifier__learning_rate': 0.001, 'classifier__max_depth': 3, 'classifier__n_estimators': 300}


c:\Users\Manel\miniconda3\envs\prepro\Lib\site-packages\xgboost\core.py:158: UserWarning:

[13:03:34] WARNING: D:\bld\xgboost-split_1724807710452\work\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.




Matriz de confusión: 

[[25 39]
 [53 27]]

Reporte de Clasificación: 

              precision    recall  f1-score   support

           0       0.32      0.39      0.35        64
           1       0.41      0.34      0.37        80

    accuracy                           0.36       144
   macro avg       0.36      0.36      0.36       144
weighted avg       0.37      0.36      0.36       144

Tiempo de entrenamiento: 0.86 segundos


In [118]:
# Definir las métricas a calcular
scoring = ['f1_macro', 'accuracy', 'roc_auc']

# Validación cruzada con múltiples métricas
cv_results = cross_validate(pipeline, X_train, y_train, cv=5, scoring=scoring, return_train_score=True)

# Mostrar resultados
print("F1-Score por fold:", cv_results['test_f1_macro'])
print("ROC-AUC por fold:", cv_results['test_roc_auc'])
print("Exactitud por fold:", cv_results['test_accuracy'])
print("Promedio F1-Score:", cv_results['test_f1_macro'].mean())

F1-Score por fold: [0.34810127 0.34810127 0.34615385 0.34615385 0.34615385]
ROC-AUC por fold: [0.5 0.5 0.5 0.5 0.5]
Exactitud por fold: [0.53398058 0.53398058 0.52941176 0.52941176 0.52941176]
Promedio F1-Score: 0.3469328140214216


### Random Forest

In [215]:
from sklearn.ensemble import RandomForestClassifier

# Suponiendo que ya tienes tus datos listos en X_train, y_train, X_test, y_test

# Definir el modelo
random_forest = RandomForestClassifier(random_state=42)

# Definir el pipeline (puedes agregar más pasos si es necesario)
pipe = Pipeline([
    ('preprocessor', preprocessor),  # Preprocesador que creaste anteriormente
    ('classifier', random_forest)    # Modelo de clasificación
])

# Entrenar y evaluar
train_and_evaluate(pipe, X_train, y_train, X_test, y_test)

Matriz de confusión: 

[[17 47]
 [52 28]]

Reporte de Clasificación: 

              precision    recall  f1-score   support

           0       0.25      0.27      0.26        64
           1       0.37      0.35      0.36        80

    accuracy                           0.31       144
   macro avg       0.31      0.31      0.31       144
weighted avg       0.32      0.31      0.31       144

Tiempo de entrenamiento: 0.49 segundos


In [218]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

# Definir el modelo RandomForestClassifier
rf_model = RandomForestClassifier(random_state=42)

# Definir el pipeline
pipe = Pipeline([
    ('preprocessor', preprocessor),  # Preprocesador
    ('classifier', rf_model)         # Modelo RandomForest
])

# Definir el espacio de búsqueda de hiperparámetros
param_grid = {
    'classifier__n_estimators': [100, 200, 300],  # Número de árboles
    'classifier__max_depth': [10, 20, 30],        # Profundidad máxima del árbol
    'classifier__min_samples_split': [2, 5, 10],  # Mínimas muestras para dividir un nodo
    'classifier__min_samples_leaf': [1, 2, 4],    # Mínimas muestras en una hoja
    'classifier__bootstrap': [True, False]        # Método de muestreo
}

# Crear el objeto GridSearchCV
grid_search = GridSearchCV(pipe, param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=2)

# Entrenar el GridSearchCV para encontrar los mejores hiperparámetros
grid_search.fit(X_train, y_train)

# Mostrar los mejores hiperparámetros encontrados
print("Mejores hiperparámetros encontrados:")
print(grid_search.best_params_)

# Evaluar el modelo con los mejores hiperparámetros en el conjunto de prueba
train_and_evaluate(grid_search.best_estimator_, X_train, y_train, X_test, y_test, print_=True)

Fitting 5 folds for each of 162 candidates, totalling 810 fits
Mejores hiperparámetros encontrados:
{'classifier__bootstrap': True, 'classifier__max_depth': 10, 'classifier__min_samples_leaf': 4, 'classifier__min_samples_split': 2, 'classifier__n_estimators': 100}
Matriz de confusión: 

[[17 47]
 [52 28]]

Reporte de Clasificación: 

              precision    recall  f1-score   support

           0       0.25      0.27      0.26        64
           1       0.37      0.35      0.36        80

    accuracy                           0.31       144
   macro avg       0.31      0.31      0.31       144
weighted avg       0.32      0.31      0.31       144

Tiempo de entrenamiento: 0.43 segundos


### SVM

In [216]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

# Definir el modelo
svm_model = SVC(probability=True, random_state=42)

# Definir el pipeline (puedes agregar más pasos si es necesario)
pipe = Pipeline([
    ('preprocessor', preprocessor),  # Preprocesador
    ('classifier', svm_model)        # Modelo SVM
])

# Entrenar y evaluar
train_and_evaluate(pipe, X_train, y_train, X_test, y_test)

Matriz de confusión: 

[[16 48]
 [54 26]]

Reporte de Clasificación: 

              precision    recall  f1-score   support

           0       0.23      0.25      0.24        64
           1       0.35      0.33      0.34        80

    accuracy                           0.29       144
   macro avg       0.29      0.29      0.29       144
weighted avg       0.30      0.29      0.29       144

Tiempo de entrenamiento: 0.12 segundos


In [219]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

# Definir el modelo SVC
svm_model = SVC(probability=True, random_state=42)

# Definir el pipeline
pipe = Pipeline([
    ('preprocessor', preprocessor),  # Preprocesador
    ('classifier', svm_model)        # Modelo SVM
])

# Definir el espacio de búsqueda de hiperparámetros
param_grid = {
    'classifier__C': [0.1, 1, 10, 100],          # Parámetro de regularización
    'classifier__kernel': ['linear', 'rbf'],     # Tipo de kernel
    'classifier__gamma': ['scale', 'auto'],      # Parámetro del kernel RBF
    'classifier__degree': [2, 3, 4]              # Solo si se usa kernel polinómico
}

# Crear el objeto GridSearchCV
grid_search = GridSearchCV(pipe, param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=2)

# Entrenar el GridSearchCV para encontrar los mejores hiperparámetros
grid_search.fit(X_train, y_train)

# Mostrar los mejores hiperparámetros encontrados
print("Mejores hiperparámetros encontrados:")
print(grid_search.best_params_)

# Evaluar el modelo con los mejores hiperparámetros en el conjunto de prueba
train_and_evaluate(grid_search.best_estimator_, X_train, y_train, X_test, y_test, print_=True)

Fitting 5 folds for each of 48 candidates, totalling 240 fits
Mejores hiperparámetros encontrados:
{'classifier__C': 1, 'classifier__degree': 2, 'classifier__gamma': 'scale', 'classifier__kernel': 'rbf'}
Matriz de confusión: 

[[16 48]
 [54 26]]

Reporte de Clasificación: 

              precision    recall  f1-score   support

           0       0.23      0.25      0.24        64
           1       0.35      0.33      0.34        80

    accuracy                           0.29       144
   macro avg       0.29      0.29      0.29       144
weighted avg       0.30      0.29      0.29       144

Tiempo de entrenamiento: 0.12 segundos
